# Think Bayes: Chapter 2

This notebook presents example code and exercise solutions for Think Bayes.

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

In [2]:
from __future__ import print_function, division

% matplotlib inline

from thinkbayes2 import Hist, Pmf, Suite

## The Pmf class

I'll start by making a Pmf that represents the outcome of a six-sided die.  Initially there are 6 values with equal probability.

In [4]:
pmf = Pmf()
for x in [1,2,3,4,5,6]:
    pmf[x] = 1
    
pmf.Print()

1 1
2 1
3 1
4 1
5 1
6 1


To be true probabilities, they have to add up to 1.  So we can normalize the Pmf:

In [5]:
pmf.Normalize()

6

The return value from `Normalize` is the sum of the probabilities before normalizing.

In [6]:
pmf.Print()

1 0.16666666666666666
2 0.16666666666666666
3 0.16666666666666666
4 0.16666666666666666
5 0.16666666666666666
6 0.16666666666666666


A faster way to make a Pmf is to provide a sequence of values.  The constructor adds the values to the Pmf and then normalizes:

In [7]:
pmf = Pmf([1,2,3,4,5,6])
pmf.Print()

1 0.16666666666666666
2 0.16666666666666666
3 0.16666666666666666
4 0.16666666666666666
5 0.16666666666666666
6 0.16666666666666666


To extract a value from a Pmf, you can use `Prob`

In [8]:
pmf.Prob(1)

0.16666666666666666

Or you can use the bracket operator.  Either way, if you ask for the probability of something that's not in the Pmf, the result is 0.

In [7]:
pmf[1]

0.16666666666666666

## The cookie problem

Here's a Pmf that represents the prior distribution.

In [8]:
pmf = Pmf()
pmf['Bowl 1'] = 0.5
pmf['Bowl 2'] = 0.5
pmf.Print()

Bowl 1 0.5
Bowl 2 0.5


And we can update it using `Mult`

In [9]:
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)
pmf.Print()

Bowl 1 0.375
Bowl 2 0.25


Or here's the shorter way to construct the prior.

In [10]:
pmf = Pmf(['Bowl 1', 'Bowl 2'])
pmf.Print()

Bowl 1 0.5
Bowl 2 0.5


And we can use `*=` for the update.

In [11]:
pmf['Bowl 1'] *= 0.75
pmf['Bowl 2'] *= 0.5
pmf.Print()

Bowl 1 0.375
Bowl 2 0.25


Either way, we have to normalize the posterior distribution.

In [12]:
pmf.Normalize()
pmf.Print()

Bowl 1 0.6
Bowl 2 0.4


## The Bayesian framework

Here's the same computation encapsulated in a class.

In [13]:
class Cookie(Pmf):
    """A map from string bowl ID to probablity."""

    def __init__(self, hypos):
        """Initialize self.

        hypos: sequence of string bowl IDs
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

    def Update(self, data):
        """Updates the PMF with new data.

        data: string cookie type
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

    mixes = {
        'Bowl 1':dict(vanilla=0.75, chocolate=0.25),
        'Bowl 2':dict(vanilla=0.5, chocolate=0.5),
        }

    def Likelihood(self, data, hypo):
        """The likelihood of the data under the hypothesis.

        data: string cookie type
        hypo: string bowl ID
        """
        mix = self.mixes[hypo]
        like = mix[data]
        return like

We can confirm that we get the same result.

In [14]:
pmf = Cookie(['Bowl 1', 'Bowl 2'])
pmf.Update('vanilla')
pmf.Print()

Bowl 1 0.6
Bowl 2 0.4


But this implementation is more general; it can handle any sequence of data.

In [15]:
dataset = ['vanilla', 'chocolate', 'vanilla']
for data in dataset:
    pmf.Update(data)
    
pmf.Print()

Bowl 1 0.627906976744
Bowl 2 0.372093023256


## The Monty Hall problem

The Monty Hall problem might be the most contentious question in
the history of probability.  The scenario is simple, but the correct
answer is so counterintuitive that many people just can't accept
it, and many smart people have embarrassed themselves not just by
getting it wrong but by arguing the wrong side, aggressively,
in public.

Monty Hall was the original host of the game show *Let's Make a
Deal*.  The Monty Hall problem is based on one of the regular
games on the show.  If you are on the show, here's what happens:

*  Monty shows you three closed doors and tells you that there is a
   prize behind each door: one prize is a car, the other two are less
   valuable prizes like peanut butter and fake finger nails.  The
   prizes are arranged at random.

*  The object of the game is to guess which door has the car.  If
   you guess right, you get to keep the car.

*  You pick a door, which we will call Door A.  We'll call the
   other doors B and C.

*  Before opening the door you chose, Monty increases the
   suspense by opening either Door B or C, whichever does not
   have the car.  (If the car is actually behind Door A, Monty can
   safely open B or C, so he chooses one at random.)

*  Then Monty offers you the option to stick with your original
   choice or switch to the one remaining unopened door.

The question is, should you "stick" or "switch" or does it
make no difference?

Most people have the strong intuition that it makes no difference.
There are two doors left, they reason, so the chance that the car
is behind Door A is 50%.

But that is wrong.  In fact, the chance of winning if you stick
with Door A is only 1/3; if you switch, your chances are 2/3.

Here's a class that solves the Monty Hall problem.

In [16]:
class Monty(Pmf):
    """Map from string location of car to probability"""

    def __init__(self, hypos):
        """Initialize the distribution.

        hypos: sequence of hypotheses
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()

    def Update(self, data):
        """Updates each hypothesis based on the data.

        data: any representation of the data
        """
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

    def Likelihood(self, data, hypo):
        """Compute the likelihood of the data under the hypothesis.

        hypo: string name of the door where the prize is
        data: string name of the door Monty opened
        """
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

And here's how we use it.

In [17]:
pmf = Monty('ABC')
pmf.Update('B')
pmf.Print()

A 0.333333333333
B 0.0
C 0.666666666667


## The Suite class

Most Bayesian updates look pretty much the same, especially the `Update` method.  So we can encapsulate the framework in a class, `Suite`, and create new classes that extend it.

Child classes of `Suite` inherit `Update` and provide `Likelihood`.  So here's the short version of `Monty`

In [18]:
class Monty(Suite):

    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

And it works.

In [19]:
pmf = Monty('ABC')
pmf.Update('B')
pmf.Print()

A 0.333333333333
B 0.0
C 0.666666666667


## The M&M problem

M&Ms are small candy-coated chocolates that come in a variety of
colors.  Mars, Inc., which makes M&Ms, changes the mixture of
colors from time to time.

In 1995, they introduced blue M&Ms.  Before then, the color mix in
a bag of plain M&Ms was 30% Brown, 20% Yellow, 20% Red, 10%
Green, 10% Orange, 10% Tan.  Afterward it was 24% Blue , 20%
Green, 16% Orange, 14% Yellow, 13% Red, 13% Brown.

Suppose a friend of mine has two bags of M&Ms, and he tells me
that one is from 1994 and one from 1996.  He won't tell me which is
which, but he gives me one M&M from each bag.  One is yellow and
one is green.  What is the probability that the yellow one came
from the 1994 bag?

Here's a solution:

In [9]:
class M_and_M(Suite):
    """Map from hypothesis (A or B) to probability."""

    mix94 = dict(brown=30,
                 yellow=20,
                 red=20,
                 green=10,
                 orange=10,
                 tan=10,
                 blue=0)

    mix96 = dict(blue=24,
                 green=20,
                 orange=16,
                 yellow=14,
                 red=13,
                 brown=13,
                 tan=0)

    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)

    hypotheses = dict(A=hypoA, B=hypoB)

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: string hypothesis (A or B)
        data: tuple of string bag, string color
        """
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

And here's an update:

In [10]:
suite = M_and_M('AB')
suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))
suite.Print()

A 0.7407407407407407
B 0.2592592592592592


**Exercise:**  Suppose you draw another M&M from `bag1` and it's blue.  What can you conclude?  Run the update to confirm your intuition.

In [11]:
suite.Update(('bag1', 'blue'))
suite.Print()

A 0.0
B 1.0


**Exercise:**  Now suppose you draw an M&M from `bag2` and it's blue.  What does that mean?  Run the update to see what happens.

In [12]:
# Solution goes here
suite.Update(('bag2', 'blue'))
suite.Print()

ValueError: Normalize: total probability is zero.

## Exercises

**Exercise:** This one is from one of my favorite books, David MacKay's "Information Theory, Inference, and Learning Algorithms":

> Elvis Presley had a twin brother who died at birth.  What is the probability that Elvis was an identical twin?"
    
To answer this one, you need some background information: According to the Wikipedia article on twins:  ``Twins are estimated to be approximately 1.9% of the world population, with monozygotic twins making up 0.2% of the total---and 8% of all twins.''

In [14]:
# Solution goes here
'''
CAUTION NOTE : use dictionary for [PRIORs] ---- hypotheses do NOT have uniform probability in the beginning

Using the fact that Elvis was a twin as background information,
hypotheses
- Elvis was an identical twin (given he was a twin)
- Elvis was not an identical twin (given he was a twin)

New Data
- Elvis' twin was Male
'''

pmf = Pmf(dict(identical=0.08, diff=0.92))

# New data = other twin was a Male (just like Elvis)...

# 100% chance that OTHER twin was Male (given they are Elvis' identical twin)
pmf["identical"] *= 1

# 50% chance that OTHER twin was Male (given they are NOT Elvis' identical twin)
pmf["diff"] *= 0.5

pmf.Normalize()
pmf.Print()

diff 0.8518518518518517
identical 0.14814814814814814


In [25]:
# Solution goes here

**Exercise:** Let's consider a more general version of the Monty Hall problem where Monty is more unpredictable.  As before, Monty never opens the door you chose (let's call it A) and never opens the door with the prize.  So if you choose the door with the prize, Monty has to decide which door to open.  Suppose he opens B with probability `p` and C with probability `1-p`.  If you choose A and Monty opens B, what is the probability that the car is behind A, in terms of `p`?  What if Monty opens C?

Hint: you might want to use SymPy to do the algebra for you. 

In [24]:
from sympy import symbols
p = symbols('p')

'''
Hypotheses
- A : Car is behind Door A
... etc

Data
- Which door Monty chooses
'''

# If Monty chooses B

pmf = Pmf('ABC')
pmf['A'] *= p # Under hypothesis that [Car in A], prob of Monty opening [B] = p 
pmf['B'] *= 0
pmf['C'] *= 1

pmf.Normalize()
pmf['A'].simplify()
#pmf.Print()


1.0*p/(p + 1)

In [26]:
# Solution goes here

# Solution if p = 0.5 (and Monty opens B)
pmf['A'].evalf(subs={'p':0.5})

0.333333333333333

In [29]:
# Solution goes here
# Solution if p = 0.0 (and Monty opens B)
# If we KNOW the Car is in C
pmf['C'].evalf(subs={'p':0.0})

1.00000000000000

In [30]:
# Solution goes here

# Solution

# And here's the solution if Monty opens C.

pmf = Pmf('ABC')
pmf['A'] *= 1-p
pmf['B'] *= 1
pmf['C'] *= 0
pmf.Normalize()
pmf['A'].simplify()

0.333333333333333*(p - 1)/(0.333333333333333*p - 0.666666666666667)

In [30]:
# Solution goes here

**Exercise:** According to the CDC, ``Compared to nonsmokers, men who smoke are about 23 times more likely to develop lung cancer and women who smoke are about 13 times more likely.''  Also, among adults in the U.S. in 2014:

> Nearly 19 of every 100 adult men (18.8%)
> Nearly 15 of every 100 adult women (14.8%)

If you learn that a woman has been diagnosed with lung cancer, and you know nothing else about her, what is the probability that she is a smoker?

In [20]:
# Solution goes here

'''
Hypotheses (Given lung cancer diagnosis as background information)
- She is a smoker
- She is NOT a smoker


"Women who smoke are about 13 times more likely... [to develop lung cancer]"
P(13/14) : female smokers, of females who develop lung cancer
P(1/14) : female nonsmokers, of females who develop lung cancer


Prior
- Female smoker : 14.8 %
- Female non-smoker : 85.2 %
'''

pmf = Pmf(dict(smoker=14.8, nonsmoker=100-14.8))

# 13 times the likelihood... due to normalizing, doesn't matter except ratio
pmf['smoker'] *= 13
pmf['nonsmoker'] *= 1

pmf.Normalize()
pmf.Print()


nonsmoker 0.3069164265129683
smoker 0.6930835734870318


**Exercise** In Section 2.3 I said that the solution to the cookie problem generalizes to the case where we draw multiple cookies with replacement.

But in the more likely scenario where we eat the cookies we draw, the likelihood of each draw depends on the previous draws.

Modify the solution in this chapter to handle selection without replacement. Hint: add instance variables to Cookie to represent the hypothetical state of the bowls, and modify Likelihood accordingly. You might want to define a Bowl object.

In [38]:
# Solution goes here

'''
Same Cookie picking class... but WITHOUT replacement!!

Bowl 1: 30V, 10C
Bowl 2: 20V, 20C

'''
class Cookie(Pmf):
    class Bowl(object):
        def __init__(self, V, C):
            self.V = V
            self.C = C
        def probV(self):
            return (self.V) / (self.V + self.C)
        def probC(self):
            return (self.C) / (self.V + self.C)
        def getLike(self, data):
            if data == 'V':
                result = self.probV()
                self.V -= 1
                return result
            else:
                result = self.probC()
                self.C -= 1
                return result
    B1 = Bowl(30,10)
    B2 = Bowl(20,20)
    mixes = {
        'Bowl 1':B1,
        'Bowl 2':B2
    }
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
    
    def Likelihood(self, data, hypo):
        b = self.mixes[hypo]
        like = b.getLike(data)
        return like

In [39]:
# Solution goes here
pmf = Cookie(['Bowl 1', 'Bowl 2'])
pmf.Update('V')
pmf.Print()

Bowl 1 0.6000000000000001
Bowl 2 0.4


In [40]:
# Solution goes here
pmf.Update('C')
pmf.Print()

Bowl 1 0.4285714285714286
Bowl 2 0.5714285714285714


In [42]:
# Solution goes here
for i in range(10):
    pmf.Update('C')
    pmf.Print()

Bowl 1 0.2621359223300971
Bowl 2 0.7378640776699028
Bowl 1 0.13636363636363635
Bowl 2 0.8636363636363635
Bowl 1 0.06104651162790699
Bowl 2 0.9389534883720931
Bowl 1 0.023800528900642243
Bowl 2 0.9761994710993579
Bowl 1 0.008061420345489446
Bowl 2 0.9919385796545106
Bowl 1 0.002316602316602318
Bowl 2 0.9976833976833978
Bowl 1 0.0005355548943766738
Bowl 2 0.9994644451056234
Bowl 1 8.929900282780178e-05
Bowl 2 0.9999107009971722
Bowl 1 8.118750253710948e-06
Bowl 2 0.9999918812497463
Bowl 1 0.0
Bowl 2 1.0


In [36]:
# Solution goes here

In [37]:
# Solution goes here

In [38]:
# Solution goes here